# Memory-Based Collaborative Filtering:
**User-Based Collaborative Filtering**

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

# Load the dataset
df = pd.read_csv('shoes-ratings.csv')

# Create a user-item matrix
user_item_matrix = df.pivot_table(index='reviewer_name', columns='url', values='review_rating', fill_value=0)

# Calculate user similarity
user_similarity = 1 - pairwise_distances(user_item_matrix, metric='correlation')


In [2]:
user_similarity

array([[ 1.00000000e+00, -9.20810313e-04, -9.20810313e-04, ...,
        -9.20810313e-04, -9.20810313e-04, -9.20810313e-04],
       [-9.20810313e-04,  1.00000000e+00, -9.20810313e-04, ...,
        -9.20810313e-04, -9.20810313e-04, -9.20810313e-04],
       [-9.20810313e-04, -9.20810313e-04,  1.00000000e+00, ...,
        -9.20810313e-04, -9.20810313e-04, -9.20810313e-04],
       ...,
       [-9.20810313e-04, -9.20810313e-04, -9.20810313e-04, ...,
         1.00000000e+00, -9.20810313e-04, -9.20810313e-04],
       [-9.20810313e-04, -9.20810313e-04, -9.20810313e-04, ...,
        -9.20810313e-04,  1.00000000e+00, -9.20810313e-04],
       [-9.20810313e-04, -9.20810313e-04, -9.20810313e-04, ...,
        -9.20810313e-04, -9.20810313e-04,  1.00000000e+00]])

In [3]:
# Function to recommend products to a user based on user similarity
def user_based_recommendation(reviewer_name, num_recommendations=5):
    # Find similar users based on user similarity
    user_id = df.index[df["reviewer_name"]==reviewer_name][0]
    similar_users = user_similarity[user_id].argsort()[::-1][1:]

    # Get the products not rated by the user
    user_rated_products = user_item_matrix.loc[reviewer_name]
    unrated_products = user_rated_products[user_rated_products == 0].index

    # Calculate the average rating of similar users for each unrated product
    product_ratings = user_item_matrix.loc[df["reviewer_name"][similar_users], unrated_products].mean()

    # Get the top recommended products
    recommended_products = product_ratings.sort_values(ascending=False).head(num_recommendations)

    return recommended_products

# Example usage

reviewer_name='bob 53'
recommendations = user_based_recommendation(reviewer_name)

print(f"Recommended products for user {reviewer_name}:")
print(recommendations)

Recommended products for user bob 53:
url
https://www.amazon.co.uk/dp/B071F6XLYX    0.342686
https://www.amazon.co.uk/dp/B07PG56S9Q    0.331644
https://www.amazon.co.uk/dp/B07KXZBJ3N    0.278059
https://www.amazon.co.uk/dp/B07KRNF4C1    0.277516
https://www.amazon.co.uk/dp/B088LGY9NK    0.262491
dtype: float64


# Model-Based Collaborative Filtering:

In [4]:
ratings_utility_matrix = df.pivot_table(values='review_rating', index='reviewer_name', columns='url', fill_value=0)
ratings_utility_matrix.head()

url,https://www.amazon.co.uk/dp/B0007T5JFK,https://www.amazon.co.uk/dp/B000BO4LCI,https://www.amazon.co.uk/dp/B000BO9WE0,https://www.amazon.co.uk/dp/B000I13RQA,https://www.amazon.co.uk/dp/B000K8FEYO,https://www.amazon.co.uk/dp/B000XB318Y,https://www.amazon.co.uk/dp/B001611GS2,https://www.amazon.co.uk/dp/B001MBLNU2,https://www.amazon.co.uk/dp/B001R0XHPC,https://www.amazon.co.uk/dp/B001UE70BW,...,https://www.amazon.co.uk/dp/B094SWK5TG,https://www.amazon.co.uk/dp/B095CJB6D8,https://www.amazon.co.uk/dp/B095J4QY9T,https://www.amazon.co.uk/dp/B096FL2FV1,https://www.amazon.co.uk/dp/B096FZBMDN,https://www.amazon.co.uk/dp/B096Y8ZCLB,https://www.amazon.co.uk/dp/B09772P1X3,https://www.amazon.co.uk/dp/B097DZV4TM,https://www.amazon.co.uk/dp/B09889G9N8,https://www.amazon.co.uk/dp/B09BN7XC4Z
reviewer_name,,,,,,,,,,,,,,,,,,,,,
Edith,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Gaby K.,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Rafsecurity,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
leonardo,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
piñeiro,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = ratings_utility_matrix.T
X.head()

reviewer_name,Edith,Gaby K.,Rafsecurity,leonardo,piñeiro,"""ratedmosquito""",./.,.93;Image,.NP.,007,...,花澤,茶色い眼鏡の男,藤谷小夏,街の遊撃手,飛んで火に入る,高橋,𝐣𝐚𝐩𝐨𝐦𝐚𝐠 ツ,💕Rosy💕,📙 GILL,🖤
url,,,,,,,,,,,,,,,,,,,,,
https://www.amazon.co.uk/dp/B0007T5JFK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://www.amazon.co.uk/dp/B000BO4LCI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://www.amazon.co.uk/dp/B000BO9WE0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://www.amazon.co.uk/dp/B000I13RQA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://www.amazon.co.uk/dp/B000K8FEYO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
import sklearn
from sklearn.decomposition import TruncatedSVD


SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(1087, 10)

In [7]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(1087, 1087)

In [8]:
X.index[1]

'https://www.amazon.co.uk/dp/B000BO4LCI'

In [9]:
i = X.index[1]

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

1

In [10]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(1087,)

In [11]:
Recommend = list(X.index[correlation_product_ID > 0.989])

# Removes the item already bought by the customer
Recommend.remove(i)

sorted(Recommend[0:9],reverse=True)

['https://www.amazon.co.uk/dp/B07FD43S85']